In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

# from cvnn import layers
# import cvnn

from pdb import set_trace
# import tensorflow_datasets as tfds
import scipy
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
import cmath
import scipy.io as sio
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.models import Model
import keras.backend as K
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Conv2DTranspose, BatchNormalization, Dropout, Input, MaxPool2D, Activation
from tensorflow.keras.layers import SimpleRNN, LSTM, TimeDistributed, Flatten
import gc
# import tensorflow_probability as tfp


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
ph_size = 100
path = "../../../Dataset/sensor_domain/high_freq/100_breast_150_40/*"
test_size = 0.3

In [ ]:
def mse_1000(y_true, y_pred):
    mse = tf.norm((y_true - y_pred))
    return mse

In [ ]:
def read_data(path, test_size=0.25):
    
    ground_truth = []
    sensor_data = []
    count = 0
    for filename in glob.glob(path):
        mat = sio.loadmat(filename)
        truth = mat["truth"]
        sensor = mat["sensor"]
        min_gt = mat["min_truth"]
        max_gt = mat["max_truth"]
        min_sensor = mat["min_sensor"]
        max_sensor = mat["max_sensor"]
        sensor_norm = mat["sensor_norm"]
        
        
        gt = (truth - min_gt) / (max_gt - min_gt)
        # gt = np.expand_dims(gt, axis=0)
        # sensor_norm = np.expand_dims(sensor_norm, axis=0)

        temp = np.append(sensor_norm.real, sensor_norm.imag)
        temp = np.reshape(temp, (ph_size*2, ph_size))
        temp = np.asarray(temp)
        ground_truth.append(gt)
        sensor_data.append(temp)
    temp_gt = []
    temp_sens = []
    for i in range(len(ground_truth)):
        if np.isnan(ground_truth[i]).any() == False:
            temp_gt.append(ground_truth[i])
            temp_sens.append(sensor_data[i])
    ground_truth = temp_gt
    sensor_data = temp_sens
    
#     ground_truth = [i for i in ground_truth if np.isnan(i).any() == False]
#     sensor_data = [i for i in sensor_data if np.isnan(i).any() == False]
    
    X_train, X_test, y_train, y_test = train_test_split(sensor_data,ground_truth, test_size=test_size, random_state=22)
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    # X_train = np.reshape(X_train, (np.asarray(X_train).shape[0], 1, np.asarray(X_train).shape[1]))
    # X_test = np.reshape(X_test, (np.asarray(X_test).shape[0], 1, np.asarray(X_test).shape[1]))
    # y_train = np.reshape(y_train, (np.asarray(y_train).shape[0], 1, np.asarray(y_train).shape[1]))
    # y_test = np.reshape(y_test, (np.asarray(y_test).shape[0], 1, np.asarray(y_test).shape[1]))
    # X_train = np.expand_dims(X_train, axis=-1)
    # X_test = np.expand_dims(X_test, axis=-1)
    # y_train = np.expand_dims(y_train, axis=-1)
    # y_test = np.expand_dims(y_test, axis=-1)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = read_data(path, test_size=test_size)

print("Data read : \nX_train = {}\ny_train = {}\nX_test = {}\ny_test = {}".format(len(X_train), len(y_train), len(X_test), len(y_test)))
X_test.shape

In [ ]:
def model_lstm():
    model = tf.keras.Sequential()
    # model.add(tf.keras.layers.Input(shape=(None,64,64)))
    model.add(LSTM(4096,return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
    # model.add(LSTM(4096))
    model.add(Dense(ph_size*ph_size))

    model.add(Reshape([ph_size, ph_size]))

    adam = tf.keras.optimizers.Adam(learning_rate=1e-4, decay=1e-9)
    sgd = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.8, nesterov=True, name="SGD")
    rmsprop = tf.keras.optimizers.RMSprop(learning_rate=0.00001, rho=0.9, momentum=0.0, epsilon=1e-07)

    model.compile(loss = 'mse', optimizer='adam',metrics = ['accuracy'])
    return model
    # model.build()

In [ ]:
base_model = model_lstm()
base_model.summary()

In [ ]:
save_best_model = tf.keras.callbacks.ModelCheckpoint("Checkpoints/model2.hdf5", monitor='val_loss', save_best_only=True, verbose=1)

def decay_lr(epoch, lr):
    if epoch % 10 == 0:
        lr = lr - (lr*0.25)
        return lr
    return lr

lr_decay = tf.keras.callbacks.LearningRateScheduler(decay_lr, verbose=1)

class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7,
                              patience=4, min_lr=1e-9, verbose=1)


In [ ]:
history = base_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[lr_decay])

In [ ]:
x=413

test = base_model.predict(np.expand_dims(X_test[x], axis=0))
plt.imshow(test.squeeze(), cmap='gray')
# plt.imsave("../3_pred.jpg", test.squeeze(), cmap='gray')

In [ ]:
plt.imshow(y_test[x].squeeze(), cmap='gray')
# plt.imsave("../3_gt.jpg", y_test[x].squeeze(), cmap='gray')

In [ ]:
plt.imshow(X_test[x].squeeze(), cmap='gray')
# plt.imsave("../3_gt.jpg", y_test[x].squeeze(), cmap='gray')

In [ ]:
temp1 = test.squeeze()
temp2 = y_test[x].squeeze()
import math
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

psnr(temp1, temp2)

In [ ]:
# base_model.save("Weights/lstm_model_1_4096_100epochs_40db_150fov_75")

## Loading the LSTM model

In [ ]:
# base_model = tf.keras.models.load_model("Weights/lstm_model_1_4096_100epochs_40db_150fov_75")
# base_model.summary()

In [ ]:
base_model = tf.keras.models.load_model("Weights/lstm_model_1_4096_100epochs")

first_model = tf.keras.Model(
    inputs = base_model.inputs,
    outputs = base_model.layers[-1].output,
)
first_model.trainable = False
second_model = tf.keras.Sequential(
    [
        first_model,
        Reshape((100,100,1)),
        Conv2D(16, kernel_size=3, activation='relu', padding='same'),
        # Conv2D(32, kernel_size=5, activation='relu', padding='same'),
        # Conv2D(64, kernel_size=5, activation='relu', padding='same'),
        Conv2D(1, kernel_size=3, activation='relu', padding='same'),
    ]
)

second_model.compile(loss = 'mse', optimizer='adam',metrics = ['accuracy'])
second_model.summary()

In [ ]:
history = second_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [ ]:


test1 = second_model.predict(np.expand_dims(X_test[x], axis=0))
plt.imshow(test1.squeeze(), cmap='gray')
# plt.imsave("../3_pred.jpg", test.squeeze(), cmap='gray')

In [ ]:
x=413
test2 = base_model.predict(np.expand_dims(X_test[x], axis=0))
plt.imshow(test2.squeeze(), cmap='gray')

In [ ]:
plt.imshow(y_test[x].squeeze(), cmap='gray')
# plt.imsave("../3_gt.jpg", y_test[x].squeeze(), cmap='gray')

In [ ]:
temp1 = test1.squeeze()
temp2 = y_test[x].squeeze()
import math
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

psnr(temp1, temp2)

In [ ]:
temp1 = test2.squeeze()
temp2 = y_test[x].squeeze()
import math
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

psnr(temp1, temp2)